You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install -q crewai==0.108.0 crewai_tools==0.38.1 langchain_community==0.3.20 langchain_ollama  ollama==0.4.7 python-dotenv
```

In [14]:
!pip install -q crewai==0.108.0 crewai_tools==0.38.1 langchain_community==0.3.20 langchain_ollama  ollama==0.4.7 python-dotenv

In [9]:
# !python -m pip install python-dotenv

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew, LLM

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from dotenv import load_dotenv

load_dotenv('creds.env')  # Adjust path if needed
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

True

In [35]:
ollama_llm = LLM(
  	model="ollama/phi4:latest",
  	api_base="http://localhost:11434"
  )



# from langchain_ollama import ChatOllama, OllamaEmbeddings

# # Initialize the ChatOllama model
# llm_local = ChatOllama(
#     model="phi4:latest",
#     base_url="http://localhost:11434/v1"
# )

# # Initialize the OllamaEmbeddings
# embeddings = OllamaEmbeddings(
#     model="nomic-embed-text:latest",
#     base_url="http://localhost:11434"
# )

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [36]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=ollama_llm,
	verbose=True
)

### Agent: Writer

In [37]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=ollama_llm,
    verbose=True
)

### Agent: Editor

In [38]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=ollama_llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [39]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [40]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [41]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [42]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [43]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 693b569d-6b0e-4c8e-ac55-9ef441d9d37d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 2b519afa-3ea0-47be-86db-84307fc05902
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 1a7eadf3-d286-4955-a4b8-998e73ff2327
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 2b519afa-3ea0-47be-86db-84307fc05902
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 1a7eadf3-d286-4955-a4b8-998e73ff2327
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1a7eadf3-d286-4955-a4b8-998e73ff2327                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 693b569d-6b0e-4c8e-ac55-9ef441d9d37d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

I now can give a great answer

Final Answer:
# Artificial Intelligence: Exploring the Latest Trends and Key Players

In today's rapidly advancing world, Artificial Intelligence (AI) has become a prominent topic of discussion. With its ability to mimic human intelligence and perform tasks that typically require human intervention, AI is revolutionizing industries and changing the way we live and work.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the advancements in Machine Learning. This technology allows machines to learn from data and improve their performance without being explicitly programmed. Natural Language Processing (NLP) is another area seeing rapid advancements, enabling machines to understand, interpret, and generate human language. Additionally, Robotic Process Automation (RPA) is streamlining business processes by automating repetitive tasks, leading to increased efficiency and cost savings.

## Key Players in the AI Industry



In [45]:
result

CrewOutput(raw="my best complete final answer to the task.  \n```markdown\n### Navigating the Future: Latest Trends and Insights in Artificial Intelligence\n\n#### Introduction\n\nArtificial intelligence (AI) is rapidly evolving, revolutionizing industries and shaping our future. For professionals, students, and enthusiasts alike, staying informed about AI trends is essential for strategic decision-making and innovation. This article offers a comprehensive overview of the latest advancements, key players, practical applications, and ethical considerations in the field of AI.\n\nUnderstanding these dynamics not only empowers individuals to make informed decisions but also encourages thoughtful engagement with emerging technologies. We will explore breakthroughs in machine learning, healthcare innovations, natural language processing, autonomous systems, and more, providing insights into how AI is transforming various sectors.\n\n#### Latest Trends in Artificial Intelligence\n\n**Machine

- Display the results of your execution as markdown in the notebook.

In [47]:
from IPython.display import Markdown
# Extract the Markdown string from the CrewOutput object
markdown_content = result.raw

# Display the Markdown content
display(Markdown(markdown_content))
# Markdown(result)

my best complete final answer to the task.  
```markdown
### Navigating the Future: Latest Trends and Insights in Artificial Intelligence

#### Introduction

Artificial intelligence (AI) is rapidly evolving, revolutionizing industries and shaping our future. For professionals, students, and enthusiasts alike, staying informed about AI trends is essential for strategic decision-making and innovation. This article offers a comprehensive overview of the latest advancements, key players, practical applications, and ethical considerations in the field of AI.

Understanding these dynamics not only empowers individuals to make informed decisions but also encourages thoughtful engagement with emerging technologies. We will explore breakthroughs in machine learning, healthcare innovations, natural language processing, autonomous systems, and more, providing insights into how AI is transforming various sectors.

#### Latest Trends in Artificial Intelligence

**Machine Learning Enhancements**: Recent advancements in deep learning algorithms have significantly improved AI's ability to analyze complex data sets. These enhancements are driving progress across multiple industries, from finance to healthcare, enabling more accurate predictions and efficient operations.

**AI in Healthcare**: Innovations such as AI-driven diagnostics and personalized medicine are transforming patient care. By leveraging vast amounts of medical data, AI systems can identify patterns and suggest treatments tailored to individual patients, improving outcomes and efficiency.

**Natural Language Processing (NLP)**: Advances in language models like GPT-4 have expanded AI's capabilities in understanding and generating human language. These developments are paving the way for more intuitive interactions between humans and machines, enhancing communication and accessibility.

**Autonomous Systems**: Progress in self-driving vehicles and robotics is accelerating, promising safer transportation and more efficient automation of repetitive tasks. As these technologies mature, they will increasingly integrate into everyday life, reshaping industries and work environments.

**Ethical AI Development**: Ensuring that AI systems are fair, transparent, and accountable remains a critical focus. Initiatives aimed at addressing ethical considerations help build trust in AI technologies and promote their responsible use across applications.

#### Key Players in the AI Landscape

**Tech Giants**: Companies like Google (DeepMind), Microsoft, Amazon, and IBM Watson continue to lead AI innovation with significant investments in research and development. Their contributions are setting industry standards and driving technological progress.

**Emerging Startups**: Smaller companies and startups bring fresh perspectives and innovative solutions to the AI landscape. These agile entities often pioneer new applications of AI technology, challenging established players and accelerating change.

**Academic Institutions**: Leading research centers such as MIT, Stanford, and Carnegie Mellon are at the forefront of AI advancements. Their contributions to foundational research and education are crucial for nurturing future generations of AI experts.

#### Noteworthy News in AI

Recent breakthroughs have captured headlines, highlighting both progress and challenges within the field. Government policies continue to shape the development and deployment of AI technologies, influencing how innovations are integrated into society. Collaborations between tech firms and public sectors further demonstrate the growing importance of AI in addressing global issues.

#### Practical Applications of AI Across Industries

**Finance**: Algorithmic trading and fraud detection systems utilize AI to enhance financial security and efficiency. These applications help institutions manage risks while optimizing investment strategies.

**Manufacturing**: Predictive maintenance and smart factories leverage AI for improved operational efficiency, reducing downtime and increasing productivity through data-driven decision-making.

**Retail**: Personalized shopping experiences are made possible by AI-driven insights into consumer behavior, enabling retailers to tailor offerings and streamline inventory management.

**Agriculture**: Precision farming and crop monitoring use AI technologies to optimize resource use and increase yields, contributing to sustainable agricultural practices.

#### Ethical Considerations and Challenges

The integration of AI raises important questions about data privacy and algorithmic bias. Ensuring transparency and fairness in AI systems is crucial for maintaining public trust. Additionally, balancing automation with employment opportunities remains a significant challenge as industries adopt more advanced technologies.

Addressing these ethical considerations requires ongoing dialogue and collaboration among stakeholders to create frameworks that prioritize human values alongside technological advancement.

#### Conclusion

The rapid evolution of artificial intelligence presents both exciting opportunities and complex challenges. Staying informed about AI trends is essential for leveraging its benefits responsibly. As the field continues to grow, engaging in continuous learning and ethical consideration will be key to navigating the future landscape of technology.

#### Call to Action

To deepen your understanding of AI advancements, consider subscribing to newsletters or joining communities focused on artificial intelligence. Engage with thought leaders and participate in discussions that explore the potential and implications of AI technologies.

By staying informed and involved, you can contribute to shaping a future where AI enhances human capabilities and fosters positive change across industries.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [48]:
topic = "Agentic AI"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 693b569d-6b0e-4c8e-ac55-9ef441d9d37d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agentic AI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 2b519afa-3ea0-47be-86db-84307fc05902
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 1a7eadf3-d286-4955-a4b8-998e73ff2327
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: de44dcd1-3216-4722-bffc-7e7ca4835c75
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 2b519afa-3ea0-47be-86db-84307fc05902
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 1a7eadf3-d286-4955-a4b8-998e73ff2327
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1a7eadf3-d286-4955-a4b8-998e73ff2327                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 693b569d-6b0e-4c8e-ac55-9ef441d9d37d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

I now can give a great answer.

Final Answer:
# Unveiling the Power of Agentic AI: Latest Trends and Key Players

## Introduction

Agentic AI, also known as autonomous AI, refers to artificial intelligence systems that have the ability to take autonomous actions and make decisions without human intervention. This technology has the potential to revolutionize various industries by streamlining processes, increasing efficiency, and enabling new opportunities for innovation. From autonomous vehicles to automated customer service chatbots, Agentic AI is reshaping the way we interact with technology.

## Latest Trends in Agentic AI

One of the key trends in Agentic AI is the advancements in machine learning algorithms. These algorithms are becoming more sophisticated and efficient, allowing AI systems to learn from data and improve their performance over time. Additionally, Agentic AI is increasingly being integrated with IoT devices, enabling seamless communication and coordination between

In [49]:

# Extract the Markdown string from the CrewOutput object
markdown_content = result.raw

# Display the Markdown content
display(Markdown(markdown_content))
# Markdown(result)
# Markdown(result)

```markdown
# A Comprehensive Overview of Agentic AI

## Introduction

Agentic AI refers to artificial intelligence systems capable of acting autonomously or semi-autonomously within a given environment. These systems are designed to perform tasks without human intervention, adapting to new situations and making decisions based on their programming and learning capabilities. The significance of agentic AI in today's technological landscape cannot be overstated; it is revolutionizing industries by enhancing efficiency and enabling innovative solutions that were previously unimaginable.

Understanding agentic AI is crucial for businesses and individuals alike. For companies, leveraging these advanced systems can lead to substantial growth opportunities, while for consumers, they offer more personalized and efficient services. As the technology evolves, staying informed about its advancements becomes essential to remain competitive and relevant in an increasingly digital world.

## Latest Trends in Agentic AI

Recent developments in agentic AI have seen remarkable integration across various industries. In healthcare, AI agents are assisting with diagnostics and patient management, improving accuracy and reducing human error. The finance sector is utilizing these systems for risk assessment and fraud detection, enhancing security measures and operational efficiency. Meanwhile, logistics companies are deploying AI to optimize supply chain operations, from inventory management to delivery routing.

The advancements in machine learning algorithms and natural language processing have been pivotal in driving these trends. These technologies enable agentic AI to process vast amounts of data more effectively, making them smarter and more adaptable. As a result, industries can harness the power of AI to solve complex problems and streamline processes, paving the way for future innovations.

## Key Players and Innovations

Leading companies like OpenAI, Google DeepMind, and IBM Watson are at the forefront of agentic AI development. These organizations have launched groundbreaking projects that demonstrate the potential of AI agents in real-world applications. For instance, OpenAI's work on language models has shown how AI can engage in human-like conversations, opening up new possibilities for customer service automation.

Industry leaders continuously push the boundaries with innovative products and solutions. As one expert noted, "The future of AI lies not just in what it can do today, but in its potential to evolve alongside us." These insights highlight the importance of ongoing research and development in shaping a smarter, more connected world.

## Noteworthy News and Breakthroughs

Recent news articles and studies have highlighted significant breakthroughs in agentic AI. A notable example is the development of AI systems capable of performing complex tasks with minimal human oversight. However, these advancements also bring challenges, such as ensuring ethical use and addressing potential biases within algorithms.

Upcoming events and conferences dedicated to AI provide platforms for sharing knowledge and fostering collaboration among experts. These gatherings are crucial for discussing future directions in agentic AI research and exploring solutions to current obstacles.

## Applications and Benefits of Agentic AI

Agentic AI finds practical applications across a wide range of sectors, from healthcare and finance to transportation and customer service. Businesses can leverage these systems to enhance operational efficiency and drive growth. For example, AI agents can automate routine tasks, allowing employees to focus on more strategic activities.

The benefits of agentic AI are manifold, including improved decision-making support, increased automation, and fostering innovation. By integrating AI into their operations, companies can not only streamline processes but also gain a competitive edge in the market.

## Ethical Considerations and Challenges

While the potential of agentic AI is immense, it raises several ethical concerns that need addressing. Issues such as privacy, bias, and accountability are at the forefront of discussions surrounding these technologies. Ensuring that AI systems operate transparently and fairly is paramount to maintaining public trust.

Regulatory measures and guidelines are being developed to address these challenges. Different perspectives on the societal impact of AI agents emphasize the importance of balancing innovation with ethical responsibility. As agentic AI continues to evolve, ongoing dialogue among stakeholders will be essential in shaping its future trajectory.

## Future Outlook

The future of agentic AI is promising, with potential advancements poised to further transform industries and society. Businesses and individuals must prepare for these changes by staying informed and adaptable. Embracing new technologies will be key to thriving in an increasingly digital world.

Encouraging readers to engage with the latest developments in AI technology can help foster a more informed community. By keeping abreast of trends and innovations, we can collectively navigate the complexities of agentic AI and harness its full potential for positive impact.

## Call to Action

We invite you to subscribe for more insights on AI technology and join discussions in our dedicated AI forums. Explore resources and tools that will enhance your understanding of agentic AI, empowering you to make informed decisions about its integration into your life or business.
```

This markdown version of the blog post is now proofread for grammatical accuracy, maintains alignment with the brand's voice, and follows journalistic best practices while providing a balanced viewpoint.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).